## Masks to Bounding boxes (semantic segmentation to object detection segmentetaion)

In [9]:
# Segmentation classes
# list_of_classes = ['Background', 'Water', 'Building_No_Damage', 'Building_Minor_Damage', 
#                    'Building_Major_Damage', 'Building_Total_Destruction', 'Vehicle', 'Road-Clear', 
#                    'Road-Blocked', 'Tree', 'Pool']

In [1]:
import cv2
import numpy as np
import os
import traceback

### Clear the saving directory

In [6]:
import shutil

def clear_directory(dir_path):
    if os.path.exists(dir_path):
        # Delete all files in the directory
        for filename in os.listdir(dir_path):
            file_path = os.path.join(dir_path, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)  # Remove the file
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)  # Remove directories and their contents
            except Exception as e:
                print(f"Failed to delete {file_path}. Reason: {e}")
    else:
        # Create the directory if it doesn't exist
        os.makedirs(dir_path)
        print(f"Created directory: {dir_path}")

### Processing Image

In [7]:
def process_image(image_path, image_paths2):
    rotated_boxes = [] # Store the rotated bounding boxes for YOLO conversion
    try:
        image1 = cv2.imread(image_path)
        image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
        
        class_colors = {
            0: (0, 0, 0),         # Background (Black)
            1: (139, 69, 19),     # Debris (SaddleBrown)
            2: (0, 191, 255),     # Water (DeepSkyBlue)
            3: (50, 205, 50),     # Building_No_Damage (LimeGreen)
            4: (255, 165, 0),     # Building_Minor_Damage (Orange)
            5: (255, 69, 0),      # Building_Major_Damage (RedOrange)
            6: (255, 0, 0),       # Building_Total_Destruction (Red)
            7: (75, 0, 130),      # Vehicle (Indigo)
            8: (169, 169, 169),   # Road (DarkGray)
            9: (34, 139, 34),     # Tree (ForestGreen)
            10: (30, 144, 255),   # Pool (DodgerBlue)
            11: (244, 164, 96)    # Sand (SandyBrown)
        }
        
        lut = np.zeros((256, 1, 3), dtype=np.uint8)
        for class_id, color in class_colors.items():
            lut[class_id] = color
        colored_image = cv2.LUT(image1, lut)

        image2 = cv2.imread(image_paths2)
        image_width = image2.shape[1]  # width
        image_height = image2.shape[0]  # height
        
        # Draw bounding boxes
        for class_id, color in class_colors.items():
            if class_id != 0:  
                mask = cv2.inRange(colored_image, np.array(color), np.array(color))
                contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                for contour in contours:
                    rotated_rect = cv2.minAreaRect(contour)
                    box = cv2.boxPoints(rotated_rect)
                    box = np.intp(box)
                    rotated_boxes.append((class_id, box))
                    rectangle_color = class_colors[class_id]
                    cv2.drawContours(image2, [box], 0, rectangle_color, 12) 
                    cv2.putText(image2, f"class:{class_id}", (box[0][0], box[0][1]), cv2.FONT_HERSHEY_SIMPLEX, 1, rectangle_color, 2, cv2.LINE_AA)

        # Save the image with bounding boxes
        save_dir = r"M:\SOL\YOLO\YOLO_RescueNet\Experimantation__Folder\Cnvrtd_train_image"
        os.makedirs(save_dir, exist_ok=True)  # Ensure the directory exists
        output_filename = os.path.basename(image_paths2)
        output_path = os.path.join(save_dir, os.path.splitext(output_filename)[0] + '_bbox.jpg')
        cv2.imwrite(output_path, image2)
    
    except Exception as e:
        print(f"Error processing {image_path} and {image_paths2}: {e}")
        traceback.print_exc()
    
    # Convert bounding boxes to YOLO format
    box_to_yolo_cords = box_to_yolo_format(rotated_boxes, image_width, image_height)
    return box_to_yolo_cords

def box_to_yolo_format(rotated_boxes, image_width, image_height):
    yolo_coordinates = []
    for r_box in rotated_boxes:
        points = r_box[1]
        points[:, 0] = np.clip(points[:, 0], 0, image_width)
        points[:, 1] = np.clip(points[:, 1], 0, image_height) 
        
        # Calculate the axis-aligned bounding box
        x_min, y_min = np.min(points, axis=0)
        x_max, y_max = np.max(points, axis=0)

        # Calculate center, width, and height
        x_center = (x_min + x_max) / 2
        y_center = (y_min + y_max) / 2
        width = x_max - x_min
        height = y_max - y_min

        # Normalize the values
        x_center_normalized = x_center / image_width
        y_center_normalized = y_center / image_height
        width_normalized = width / image_width
        height_normalized = height / image_height

        # YOLO format coordinates
        yolo_coordinate = [x_center_normalized, y_center_normalized, width_normalized, height_normalized]
        yolo_coordinates.append(yolo_coordinate)
    
    return yolo_coordinates

def main():
    # Clear saving directory
    save_dir = r"M:\SOL\YOLO\YOLO_RescueNet\Experimantation__Folder\Cnvrtd_train_image"
    clear_directory(save_dir)  # Clear the save directory before processing images
    
    image_dir = r"M:\SOL\YOLO\YOLO_RescueNet\Experimantation__Folder\Experiment_cnvrtng_labeled"
    image_dir2 = r"M:\SOL\YOLO\YOLO_RescueNet\Experimantation__Folder\Experiment_cnvrtng_org"
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir)]
    image_paths2 = [os.path.join(image_dir2, filename) for filename in os.listdir(image_dir2)]
    combined_paths = list(zip(image_paths, image_paths2))
    
    print(f"Found {len(combined_paths)} image pairs to process.")
    for image_path, image_paths2 in combined_paths:
        box_to_cord = process_image(image_path, image_paths2)
        # print(box_to_cord)

if __name__ == '__main__':
    main()

Found 82 image pairs to process.
